# **Ball by Ball data**

## Loading and Pre-Processing

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Read the CSV file into a DataFrame, setting low_memory=False to ensure all data types are inferred correctly
df = pd.read_csv('ball_by_ball_data.csv', low_memory=False)

# Display the first few rows of the DataFrame
df.head()

In [ ]:
df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed'],
      dtype='object')

In [ ]:
# Check for missing values in each column and sum them up
missing_values = df.isnull().sum()

# Display the count of missing values in each column
missing_values

match_id                       0
season                         0
start_date                     0
venue                          0
innings                        0
ball                           0
batting_team                   0
bowling_team                   0
striker                        0
non_striker                    0
bowler                         0
runs_off_bat                   0
extras                         0
wides                     187753
noballs                   192848
byes                      193107
legbyes                   190506
penalty                   193615
wicket_type               184095
player_dismissed          184095
other_wicket_type         193617
other_player_dismissed    193617
dtype: int64

In [ ]:
# Define a list of columns that are not required
unwanted_cols = ["venue", "striker", "non_striker", "bowler", "player_dismissed", "other_wicket_type",
                 "other_player_dismissed", "penalty", "byes", "legbyes"]

# Drop the unwanted columns from the DataFrame
df.drop(unwanted_cols, axis=1, inplace=True)

## Target Calculation

### adding new column "season"

In [ ]:
# Copy the 'start_date' column to a new column named 'date'
df["date"] = df["start_date"]

# Initialize an empty dictionary to store match_id and corresponding start_date
date = {}

# Iterate through each row index in the DataFrame
for i in df.index:
    # Store the start_date corresponding to match_id in the dictionary
    date[df.loc[i, "match_id"]] = df.loc[i, "start_date"]

# Convert the 'start_date' column to datetime format
df['start_date'] = pd.to_datetime(df['start_date'])

# Extract the year from the start_date and add it as a new column named 'season' in the DataFrame
df['season'] = pd.DatetimeIndex(df['start_date']).year

In [ ]:
# Drop the 'start_date' column from the DataFrame
df.drop(['start_date'], axis=1, inplace=True)

# Display the columns of the DataFrame after dropping 'start_date'
df.columns

### calculating score per ball

In [ ]:
# Initialize an empty list to store the total runs for each row
total = []

# Iterate through each row index in the DataFrame
for i in range(len(df)):
    # Calculate the total runs by adding runs_off_bat and extras for each row
    total.append(int(df.loc[i, "runs_off_bat"]) + int(df.loc[i, "extras"]))

# Add a new column named 'total' to the DataFrame with ball-wise total runs
df = df.assign(total=total)

In [ ]:
df.head()

,match_id,season,innings,ball,batting_team,bowling_team,runs_off_bat,extras,wides,noballs,byes,legbyes,wicket_type,date,total
0,335982,2008,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,NaN,NaN,NaN,1.0,NaN,2008-04-18,1
1,335982,2008,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,0,0,NaN,NaN,NaN,NaN,NaN,2008-04-18,0
2,335982,2008,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,1.0,NaN,NaN,NaN,NaN,2008-04-18,1
3,335982,2008,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,0,0,NaN,NaN,NaN,NaN,NaN,2008-04-18,0
4,335982,2008,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,0,0,NaN,NaN,NaN,NaN,NaN,2008-04-18,0


### calculating score per innings ( target )

In [ ]:
# Get unique match IDs
MATCHES = df["match_id"].unique()

# Initialize an empty dictionary to store targets for each match
targets = {}

# Iterate through each unique match ID
for match_id in MATCHES:
    # Calculate the total runs for the first innings of the match and store it in the dictionary
    targets[match_id] = df.loc[(df['match_id'] == match_id) & (df['innings'] == 1), "total"].sum()

# Print the dictionary containing targets for each match
print(targets)

In [ ]:
# Create a new column 'Targets' initialized with zeros
df["Targets"] = np.zeros(len(df))

# Iterate through each row in the DataFrame
for i in range(len(df)):
    # Update the 'Targets' column with the target value for the corresponding match_id + 1
    df.at[i, 'Targets'] = targets[df.at[i, 'match_id']] + 1

# Display the first few rows of the DataFrame with the updated 'Targets' column
df.head()

In [ ]:
#veryfication of a random match id target
df.loc[df["match_id"]==1216514]

,match_id,season,innings,ball,batting_team,bowling_team,runs_off_bat,extras,wides,noballs,byes,legbyes,wicket_type,date,total,Targets
182533,1216514,2020,1,0.1,Rajasthan Royals,Royal Challengers Bangalore,0,0,NaN,NaN,NaN,NaN,NaN,2020-10-03,0,155.0
182534,1216514,2020,1,0.2,Rajasthan Royals,Royal Challengers Bangalore,4,0,NaN,NaN,NaN,NaN,NaN,2020-10-03,4,155.0
182535,1216514,2020,1,0.3,Rajasthan Royals,Royal Challengers Bangalore,1,0,NaN,NaN,NaN,NaN,NaN,2020-10-03,1,155.0
182536,1216514,2020,1,0.4,Rajasthan Royals,Royal Challengers Bangalore,0,0,NaN,NaN,NaN,NaN,NaN,2020-10-03,0,155.0
182537,1216514,2020,1,0.5,Rajasthan Royals,Royal Challengers Bangalore,0,0,NaN,NaN,NaN,NaN,NaN,2020-10-03,0,155.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182766,1216514,2020,2,18.3,Royal Challengers Bangalore,Rajasthan Royals,0,0,NaN,NaN,NaN,NaN,NaN,2020-10-03,0,155.0
182767,1216514,2020,2,18.4,Royal Challengers Bangalore,Rajasthan Royals,1,0,NaN,NaN,NaN,NaN,NaN,2020-10-03,1,155.0
182768,1216514,2020,2,18.5,Royal Challengers Bangalore,Rajasthan Royals,1,0,NaN,NaN,NaN,NaN,NaN,2020-10-03,1,155.0
182769,1216514,2020,2,18.6,Royal Challengers Bangalore,Rajasthan Royals,1,0,NaN,NaN,NaN,NaN,NaN,2020-10-03,1,155.0


**Section Highlights**

- Loaded cricket match data from a CSV file into a DataFrame, ensuring accurate data type inference.
- Cleaned the DataFrame by dropping unnecessary columns.
- Processed date information to extract the year as the 'season'.
- Calculated total runs for each ball and added a new column named 'total'.
- Derived targets for each match from the total runs of the first innings.
- Added and updated a column 'Targets' with the derived target values.

# **Match by Match data**

### Setting Winners

In [ ]:
# Load the Matches.csv file into a DataFrame
matches = pd.read_csv("match_by_match_data.csv")

# Display the first few rows of the DataFrame to understand its structure
matches.head()

In [ ]:
# Initialize an empty dictionary to store match winners
match_winner = {}

# Populate the match_winner dictionary with match IDs as keys and corresponding winners as values
for i in matches.index:
    match_winner[matches.loc[i,'id']] = matches.loc[i,"winner"]

In [ ]:
# Initialize a list to store winners for each match in df
winners_list = []

# Iterate through rows in df to extract winner information based on match ID from match_winner dictionary
for i in df.index:
    # Append the winner for the corresponding match ID to the winners_list
    winners_list.append(match_winner[df.loc[i,"match_id"]])

# Add a new column named "winner" to df and populate it with the winners_list
df["winner"] = winners_list

# Display the first few rows of the DataFrame df with the added "winner" column
df.head()

193617


## Anomaly Removal

### Abandoned Matches

In [ ]:
# Selecting match IDs where neither batting team nor bowling team is the winner
abandoned_matches = df.loc[(df["batting_team"] != df["winner"]) & (df['bowling_team'] != df["winner"])]["match_id"].unique()

# List to store indices of rows belonging to abandoned matches
indices_to_drop = []

# Iterating over each abandoned match ID
for match_id in abandoned_matches:
    # Extending the list of indices with indices of rows belonging to the current abandoned match
    indices_to_drop.extend(list(df.loc[df["match_id"] == match_id].index))

# Dropping rows corresponding to abandoned matches
df = df.drop(indices_to_drop)

# Displaying the first few rows of the modified DataFrame
df.head()

In [ ]:
# Selecting match IDs where neither batting team nor bowling team is the winner
abandoned_matches = df.loc[(df["batting_team"] != df["winner"]) & (df['bowling_team'] != df["winner"])]["match_id"].unique()

# Displaying the match IDs of abandoned matches without any result
abandoned_matches

array([], dtype=int64)

### Super Overs

In [ ]:
# Dropping the data of the first innings
df = df.drop(list(df.loc[df["innings"] == 1].index))

# List to store match IDs that ended up in a super over
super_over_ids = []

# Searching records of matches that ended up in a super over
for i in range(3, 7):
    super_over_ids.extend(list(df.loc[df["innings"] == i]["match_id"]))

# Printing total number of matches that went up to a super over
print(len(set(super_over_ids)), "matches went up to a super over")
# Printing super over match IDs
print(set(super_over_ids))


In [ ]:
# Removing data corresponding to matches that ended up in a super over
for i in super_over_ids:
    df = df.drop(df.loc[df["match_id"] == i].index)

# Checking for the existence of super over matches
for i in super_over_ids:
    print(len(df.loc[df["match_id"] == i]), end="")

# Now, the DataFrame contains data excluding matches that went into a super over and excluding data from the first innings.


**Section Highlights**

- Extracted winner information for each match from the 'Matches.csv' file and added it to the main DataFrame.
- Identified and removed data related to abandoned matches where neither the batting nor bowling team won.
- Filtered out matches that went into a super over and removed their data from the DataFrame.
- Prepared the DataFrame for analysis by excluding data from abandoned and super over matches.

# Creating Final Columns of the data set

### Column : Runs to be scored

In [ ]:
def cum_sum(x, y):
    remaining = {}
    # Iterate through each unique match ID
    for match_id in sorted(df["match_id"].unique()):
        # Calculate the cumulative sum of column x for each match_id
        remaining[match_id] = pd.Series(df.loc[df["match_id"] == match_id, x]).cumsum()
        # Assign the cumulative sum values to column y for the corresponding match_id
        df.loc[df.match_id == match_id, y] = remaining[match_id]

In [ ]:
cum_sum("total","runs_scored")

In [ ]:
# Create a new column named 'runs_to_be_scored' by subtracting 'runs_scored' from 'Targets'
df["runs_to_be_scored"] = df["Targets"] - df["runs_scored"]

In [ ]:
# Drop the columns 'innings', 'Targets', and 'runs_scored' from the DataFrame
df.drop(columns=["innings", "Targets", "runs_scored"], inplace=True)

In [ ]:
df.head()

,match_id,season,ball,batting_team,bowling_team,runs_off_bat,extras,wides,noballs,wicket_type,date,total,winner,runs_to_be_scored
124,335982,2008,0.1,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,NaN,NaN,NaN,2008-04-18,1,Kolkata Knight Riders,222.0
125,335982,2008,0.2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,1.0,NaN,NaN,2008-04-18,1,Kolkata Knight Riders,221.0
126,335982,2008,0.3,Royal Challengers Bangalore,Kolkata Knight Riders,0,0,NaN,NaN,NaN,2008-04-18,0,Kolkata Knight Riders,221.0
127,335982,2008,0.4,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,NaN,NaN,NaN,2008-04-18,1,Kolkata Knight Riders,220.0
128,335982,2008,0.5,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,NaN,NaN,NaN,2008-04-18,1,Kolkata Knight Riders,219.0


## Balls Remaining

In [ ]:
# Initialize a new column 'balls_remaining' with initial value 1
df["balls_remaining"] = 1

# Update 'balls_remaining' column using cumulative sum function cum_sum
cum_sum("balls", "balls_remaining")

In [ ]:
# Fill missing values in 'wides', 'noballs', and 'wicket_type' columns with "0"
df["wides"].fillna("0", inplace=True)
df["noballs"].fillna("0", inplace=True)
df["wicket_type"].fillna("0", inplace=True)

In [ ]:
# Convert 'wides' and 'noballs' columns to numeric type
df["wides"] = pd.to_numeric(df["wides"])
df["noballs"] = pd.to_numeric(df["noballs"])

In [ ]:
# Create a new column 'extras' containing the maximum value of 'wides' and 'noballs' for each row
df["extras"] = df[["wides", "noballs"]].max(axis=1)

# Update 'extras' column using cumulative sum function cum_sum
cum_sum("extras", "extras")

In [ ]:
# Calculate the remaining balls for each row by subtracting 'balls' from 120 and adding 'extras'
df["balls_remaining"] = 120 - df["balls"] + df["extras"]

In [ ]:
df.loc[df["match_id"]==1216534].head()

,match_id,season,ball,batting_team,bowling_team,runs_off_bat,extras,wides,noballs,wicket_type,date,total,winner,runs_to_be_scored,balls_remaining,balls
179682,1216534,2020,0.1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,0.0,0.0,0.0,0,2020-09-21,0,Royal Challengers Bangalore,164.0,119.0,1.0
179683,1216534,2020,0.2,Sunrisers Hyderabad,Royal Challengers Bangalore,4,0.0,0.0,0.0,0,2020-09-21,4,Royal Challengers Bangalore,160.0,118.0,2.0
179684,1216534,2020,0.3,Sunrisers Hyderabad,Royal Challengers Bangalore,2,0.0,0.0,0.0,0,2020-09-21,2,Royal Challengers Bangalore,158.0,117.0,3.0
179685,1216534,2020,0.4,Sunrisers Hyderabad,Royal Challengers Bangalore,0,0.0,0.0,0.0,0,2020-09-21,0,Royal Challengers Bangalore,158.0,116.0,4.0
179686,1216534,2020,0.5,Sunrisers Hyderabad,Royal Challengers Bangalore,0,0.0,0.0,0.0,0,2020-09-21,0,Royal Challengers Bangalore,158.0,115.0,5.0


## Wickets Remaining

In [ ]:
# Replace non-zero values in 'wicket_type' column with 1
df.loc[df.wicket_type != "0", "wicket_type"] = 1

# Convert 'wicket_type' column to numeric type
df["wicket_type"] = pd.to_numeric(df["wicket_type"])

In [ ]:
cum_sum("wicket_type","wicket_type")

In [ ]:
# Calculate the remaining wickets for each row by subtracting 'wicket_type' from 10
df["wickets_remaining"] = 10 - df["wicket_type"]

In [ ]:
df.head()

,match_id,season,ball,batting_team,bowling_team,runs_off_bat,extras,wides,noballs,wicket_type,date,total,winner,runs_to_be_scored,balls_remaining,balls,wickets_remaining
124,335982,2008,0.1,Royal Challengers Bangalore,Kolkata Knight Riders,1,0.0,0.0,0.0,0.0,2008-04-18,1,Kolkata Knight Riders,222.0,119.0,1.0,10.0
125,335982,2008,0.2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1.0,1.0,0.0,0.0,2008-04-18,1,Kolkata Knight Riders,221.0,119.0,2.0,10.0
126,335982,2008,0.3,Royal Challengers Bangalore,Kolkata Knight Riders,0,1.0,0.0,0.0,0.0,2008-04-18,0,Kolkata Knight Riders,221.0,118.0,3.0,10.0
127,335982,2008,0.4,Royal Challengers Bangalore,Kolkata Knight Riders,1,1.0,0.0,0.0,0.0,2008-04-18,1,Kolkata Knight Riders,220.0,117.0,4.0,10.0
128,335982,2008,0.5,Royal Challengers Bangalore,Kolkata Knight Riders,1,1.0,0.0,0.0,0.0,2008-04-18,1,Kolkata Knight Riders,219.0,116.0,5.0,10.0


In [ ]:
# Create a new column 'new_winner' and assign value 1 where batting_team is not equal to winner, and 2 where batting_team is equal to winner
df.loc[df.batting_team != df.winner, "new_winner"] = 1
df.loc[df.batting_team == df.winner, "new_winner"] = 2

**Section Highlights**

*Runs to be Scored:*

- Created a new column named 'runs_to_be_scored' by subtracting 'runs_scored' from 'Targets'.
- Removed columns 'innings', 'Targets', and 'runs_scored' from the DataFrame.

*Balls Remaining:*

- Initialized and updated a new column 'balls_remaining' using cumulative sum calculations based on the 'balls' column.
- Filled missing values in 'wides', 'noballs', and 'wicket_type' columns with "0" and converted them to numeric type.
- Created a new column 'extras' containing the maximum value of 'wides' and 'noballs' for each row and updated it using cumulative sum calculations.
- Calculated the remaining balls for each row by subtracting 'balls' from 120 and adding 'extras'.

*Wickets Remaining:*

- Converted non-zero values in the 'wicket_type' column to 1 and calculated the remaining wickets for each row.
- Created a new column 'new_winner' with values 1 where the batting team is not equal to the winner and 2 where they are equal.

## Save Final Data Set

In [ ]:
# Create a new DataFrame 'IPL_DATASET' containing selected columns
IPL_DATASET = df.loc[:, ["date", "match_id", "runs_to_be_scored", "balls_remaining", "wickets_remaining", "new_winner"]]

In [ ]:
# Save the IPL_DATASET DataFrame to a CSV file named "final_dataset.csv"
IPL_DATASET.to_csv("dataset_before_clustering.csv", index=False)